In [1]:
import sys
sys.path.append('/workspace/Storage_fast/AneurysmSegmentation/aneurysm_segmentation3d/')
sys.path.append('/workspace/Storage_fast/AneurysmSegmentation/')


import numpy as np
import pandas as pd
import time 
import os
import torch

from omegaconf import OmegaConf
import ipdb

# Visualisation
from torch_points3d.visualization import Visualizer
import open3d as o3d
import pyvista as pv
import matplotlib.pyplot as plt
import seaborn as sns

# Torch files
from torch_points3d.datasets.dataset_factory import (
    instantiate_dataset,
)
from torch_points3d.models.model_factory import instantiate_model

# Import BaseModel / BaseDataset for type checking
from torch_points3d.models.base_model import BaseModel
from torch_points3d.datasets.base_dataset import BaseDataset
from aneurysm_segmentation3d.scripts.dataset import AneurysmDataset


# Import from metrics
from torch_points3d.metrics.base_tracker import BaseTracker
from torch_points3d.metrics.colored_tqdm import Coloredtqdm as Ctq
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

# Utils import
import shutil
from torch_points3d.utils.colors import COLORS
from aneurysm_segmentation3d.scripts.utils.wandb_utils import Wandb
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from plyfile import PlyData, PlyElement,PlyProperty, PlyListProperty


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline
sns.set(rc={'figure.figsize':(8,6)})


In [4]:
BASE_DIR = "/workspace/Storage_fast/AneurysmSegmentation/aneurysm_segmentation3d"
CONF_DIR = os.path.join(BASE_DIR, "conf/kpconv_v2_base.yaml")
DATAROOT = os.path.join(BASE_DIR, "datasets/data")
PROCESSED_DIR = os.listdir(
    os.path.join(DATAROOT, "aneurysm/processed")
)

# Copy Dataset file - for ModelCheckpoint reconstruction
SRC_DATA_PY = os.path.join(
    BASE_DIR, "scripts/dataset/AneurysmDataset.py"
)
DST_DATA_PY = "/opt/conda/envs/torchpoint/lib/python3.7/site-packages/torch_points3d/datasets/segmentation"
shutil.copy(SRC_DATA_PY, DST_DATA_PY)

# Load Config file
params = OmegaConf.load(CONF_DIR)
params.dataroot = DATAROOT
# params.parts_to_segment = PARTS_TO_SEGMENT

# Copy Config for wandb upload
SRC_CONFIG = CONF_DIR
DST_CONFIG = os.path.join(BASE_DIR, "conf/config_run.yaml")
params.config_run_path = DST_CONFIG  # make entry in config
shutil.copy(SRC_CONFIG, DST_CONFIG)

'/workspace/Storage_fast/AneurysmSegmentation/aneurysm_segmentation3d/conf/config_run.yaml'

In [5]:
dataset = AneurysmDataset.AneurysmDataset(params)
dataset

Dataset: AneurysmDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
])
test_transform = None
train_transform = Compose([
    RandomNoise(sigma=0.02, clip=0.05),
    RandomScaleAnisotropic([0.9, 1.1]),
])
val_transform = None
inference_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
])
Size of train_dataset = 73
Size of test_dataset = 12
Size of val_dataset = 25
Batch size = None

In [6]:
params.features_to_include

{'mean_curvature': 0, 'gauss_curvature': 1, 'fpfh': 1, 'shot': 0, 'rf': 0, 'ones': 1}

In [7]:
dataset.train_dataset, dataset.val_dataset, dataset.test_dataset

(Aneurysm(73), Aneurysm(25), [Aneurysm(12)])

In [8]:
dataset.train_dataset[0]

Data(category=[10437], grid_size=[1], id_scan=[1], origin_id=[10437], pos=[10437, 3], x=[10437, 4], y=[10437])

In [9]:
dataset.test_dataset[0][0]

Data(category=[11150], grid_size=[1], id_scan=[1], origin_id=[11150], pos=[11150, 3], x=[11150, 4], y=[11150])

In [ ]:
pd.Series(dataset.train_dataset[0].y.numpy()).value_counts()

0    8378
1    1898
2     139
3      22
dtype: int64

In [ ]:
dataset.train_dataset[0].pos.numpy()

array([[-0.02765902,  0.37045047, -0.67588264],
       [-0.09068815,  0.39467782, -0.64123595],
       [-0.02984019,  0.38473433, -0.6321291 ],
       ...,
       [-0.5370397 , -0.12845623,  1.033711  ],
       [-0.4963175 , -0.11800084,  1.0221816 ],
       [-0.5095957 , -0.10444126,  0.96343964]], dtype=float32)

In [ ]:
np.array(dataset.train_dataset[0].pos.numpy()).shape

(10437, 3)

In [ ]:
np.array([(0, 0, 0),
             (0, 1, 1),
             (1, 0, 1),
             (1, 1, 0)]).shape

(4, 3)

In [ ]:
a  = np.array(dataset.train_dataset[0].pos.numpy())
a

array([[-0.02026132,  0.3949881 , -0.592001  ],
       [-0.03396651,  0.38556507, -0.6078084 ],
       [-0.06574768,  0.37441754, -0.57540816],
       ...,
       [-0.56672096, -0.12089948,  0.94973636],
       [-0.5767154 , -0.13580056,  0.95919025],
       [-0.5594566 , -0.12467939,  0.95338905]], dtype=float32)

In [ ]:
#should be a list for adding  https://stackoverflow.com/questions/30062429/how-to-get-every-first-element-in-2-dimensional-list
list(list(zip(*a))[0])

[0.0056771757,
 -0.048754293,
 -0.07441491,
 -0.04349217,
 -0.006337656,
 -0.087346375,
 -0.00015158167,
 -0.014149044,
 0.043052167,
 -0.14507525,
 -0.10945931,
 -0.072001055,
 -0.095669426,
 -0.037125207,
 -0.0017817136,
 0.073635064,
 -0.14357282,
 -0.101070926,
 -0.050977856,
 -0.07844968,
 0.0087639345,
 0.012466573,
 0.021812726,
 -0.0900283,
 -0.113372035,
 -0.10570737,
 -0.08528408,
 -0.0071112695,
 -0.013898446,
 -0.031413723,
 0.005787752,
 -0.11168381,
 -0.05418161,
 -0.09005753,
 -0.03735245,
 -0.04804429,
 0.005494777,
 0.01018779,
 -0.11926645,
 -0.13611425,
 -0.0782488,
 -0.09927079,
 -0.06399247,
 -0.020390268,
 -0.009274254,
 -0.12854004,
 -0.13313365,
 -0.099161655,
 -0.08154531,
 -0.08437256,
 -0.052881587,
 -0.042154226,
 -0.007211016,
 -0.15108094,
 -0.13540804,
 -0.14875948,
 -0.10216791,
 -0.042622022,
 -0.022271845,
 0.0035648353,
 -0.005990858,
 -0.14594242,
 -0.14175466,
 -0.1060854,
 -0.0902115,
 -0.086358935,
 -0.0652888,
 -0.03844331,
 0.008362929,
 -0.1369

In [10]:

data_sample  = np.array(dataset.train_dataset[0].pos.numpy())
x = list(list(zip(*data_sample))[0])
y = list(list(zip(*data_sample))[1])
z = list(list(zip(*data_sample))[2])


a = np.empty( data_sample.shape[0],  dtype=[('x', 'f4'), ('y', 'f4'),('z', 'f4')] )

a['x'] = x
a['y'] = y
a['z'] = z

# Recreate the PlyElement instance
vert = PlyElement.describe(a, 'vertex')
# Redefine properties - to save correct file format
vert.properties = (PlyProperty('x', 'f4'),
                   PlyProperty('y', 'f4'),
                   PlyProperty('z', 'f4'),
                    )
ply_to_plot = PlyData([vert], text=True)
ply_to_plot.write('plot_file.ply')
ply_to_plot

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=10437, comments=[]),), text=True, byte_order='=', comments=[], obj_info=[])

In [11]:
ply_to_plot.elements[0].data

array([(-0.03203732,  0.4105267 , -0.6461488),
       (-0.0496915 ,  0.42641497, -0.6689632),
       (-0.04264448,  0.45294932, -0.6533574), ...,
       (-0.55010414, -0.14607765,  0.9844518),
       (-0.5312067 , -0.08943467,  0.9455886),
       (-0.5459069 , -0.11802924,  0.9732167)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [ ]:
o3d.visualization.draw_geometries(np.array(ply_to_plot.elements[0].data))
#TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
 #   1. (geometry_list: List[open3d.open3d_pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None

TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: List[open3d.open3d_pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None

Invoked with: array([(-0.0124612 ,  0.40839005, -0.6317543 ),
       (-0.05654476,  0.41481265, -0.63694966),
       (-0.00674947,  0.41598105, -0.6586964 ), ...,
       (-0.5972232 , -0.13710931,  1.0298082 ),
       (-0.5307208 , -0.14315629,  0.98350227),
       (-0.5885767 , -0.12164415,  1.0320481 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [12]:
pc_o3d = o3d.io.read_point_cloud(os.path.join(os.getcwd(),"plot_file.ply"))
o3d.visualization.draw_geometries([pc_o3d])
#RuntimeError: [Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing


RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m

In [13]:
pc_o3d = o3d.io.read_point_cloud(os.path.join(os.getcwd(),"plot_file.ply"))
o3d.visualization.draw_geometries(np.array(pc_o3d.points))
#TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
#1. (geometry_list: List[open3d.open3d_pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None

TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: List[open3d.open3d_pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None

Invoked with: array([[-0.03203732,  0.41052669, -0.6461488 ],
       [-0.0496915 ,  0.42641497, -0.66896319],
       [-0.04264448,  0.45294932, -0.65335739],
       ...,
       [-0.55010414, -0.14607765,  0.98445177],
       [-0.53120673, -0.08943467,  0.94558859],
       [-0.5459069 , -0.11802924,  0.97321671]])

In [14]:
pv.start_xvfb()

In [15]:
from pyntcloud import PyntCloud
pyntcld = PyntCloud.from_file(os.path.join(os.getcwd(),"plot_file.ply"))
pyntcld.plot()
#TypeError: "use_panel" is an invalid keyword argument for `show`

TypeError: "use_panel" is an invalid keyword argument for `show`

In [ ]:
pdata = pv.PolyData(data_sample)
sphere = pv.Sphere(radius=0.02)
pc = pdata.glyph(scale=False, geom=sphere)
pc.plot(background='black', cmap='Reds', show_scalar_bar=False)


Error: Session cannot generate requests

In [1]:
import pyvista as pv
import numpy as np

pv.start_xvfb()

point_cloud = np.random.random((100, 3))
pdata = pv.PolyData(point_cloud)
pdata['orig_sphere'] = np.arange(100)

# create many spheres from the point cloud
sphere = pv.Sphere(radius=0.02)
pc = pdata.glyph(scale=False, geom=sphere)
pc.plot(background='black', cmap='Reds', show_scalar_bar=False)

In [20]:
import pyvista as pv
sphere = pv.Sphere()
pv.start_xvfb()

p = pv.Plotter()
p.add_mesh(sphere)
p.show(auto_close=False)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [22]:
import pyvista as pv
pv.start_xvfb()

# create a mesh and identify some scalars you wish to plot
mesh = pv.Sphere()
z = mesh.points[:, 2]

# Plot using the ITKplotter
pl = pv.PlotterITK()
pl.add_mesh(mesh, scalars=z, smooth_shading=True)
pl.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [26]:
sample = dataset.train_dataset[0]
point_cloud = pv.PolyData(sample.pos.numpy())
point_cloud['y'] = sample.y.numpy()

p = pv.PlotterITK()
p.add_points(point_cloud)
# p.add_points(point_cloud,  show_scalar_bar=False, point_size=3)
p.show(True)


Viewer(geometries=[], point_set_colors=array([[1., 1., 1.]], dtype=float32), point_set_opacities=array([1.], d…

Viewer(geometries=[], point_set_colors=array([[1., 1., 1.]], dtype=float32), point_set_opacities=array([1.], d…

In [4]:
cat_to_seg = {'aneur': [0, 1, 2, 3]}
for i, seg in enumerate(cat_to_seg.values()):
    print(i,seg)

0 [0, 1, 2, 3]


In [7]:
cat_to_seg.values()

dict_values([[0, 1, 2, 3]])

In [26]:
import torch
_cat_to_seg = {}
_num_categories = len(cat_to_seg)
_max_seg_count = 0
_max_seg = 0
_shifts = torch.zeros((_num_categories,), dtype=torch.long)

for i, seg in enumerate(cat_to_seg.values()):
    _max_seg_count = max(_max_seg_count, len(seg))
    _max_seg = max(_max_seg, max(seg))
    _shifts[i] = min(seg)
    _cat_to_seg[i] = seg
    
print(_max_seg_count, _max_seg, _shifts, _cat_to_seg)

4 3 tensor([0]) {0: [0, 1, 2, 3]}


In [15]:
print(_max_seg_count, _max_seg, _shifts, _cat_to_seg)

4 3 tensor([0]) {0: [0, 1, 2, 3]}


In [ ]:
for i, seg in enumerate(dataset.class_to_segments.values()): print(i,seg)